In [1]:
import sys
sys.path.insert(1, '../../libs')
from utils import temporal_train_test_split
from autorship import AuthorClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("../../data/authors_word2vec.csv").drop(["Unnamed: 0", "comment"], axis=1)
data.head()

,username,created_utc,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,Manada_2,2022-03-09 14:17:46+00:00,-0.432714,0.032157,1.099305,0.864218,0.837875,-3.109633,0.559502,3.881276,...,2.906217,0.421129,1.411148,1.064761,1.959023,1.601933,0.523119,0.897019,-0.426631,0.103120
1,BluePirate89,2022-03-31 16:00:06+00:00,0.125898,-0.242185,0.054639,0.735328,0.204172,-0.287813,0.222079,0.502998,...,0.112329,-0.015934,0.373650,0.769785,0.104709,0.382138,-0.123530,0.540850,-0.093899,-0.297840
2,9-Cortes,2021-11-29 02:08:23+00:00,-1.789684,0.469343,0.480798,1.275060,1.201183,-4.118136,0.754351,5.680286,...,2.970081,1.143064,0.718834,0.835258,3.357937,2.351601,1.128088,-0.537648,-0.405631,-0.016753
3,Supermunch2000,2021-12-21 12:01:05+00:00,0.515221,-0.989140,0.209595,3.852181,0.815947,-1.457306,1.570920,3.043237,...,0.800782,-0.175123,1.634715,3.569113,1.248902,2.070493,-0.804131,2.869144,-0.717174,-1.667274
4,TheGza1,2021-08-10 12:49:03+00:00,-1.702236,0.606132,0.567707,1.606782,1.322883,-4.053791,0.866824,5.870177,...,3.181649,0.857197,0.913484,0.512868,3.121929,2.340846,1.325590,-0.597204,-0.446730,0.198760


In [3]:
estimators = [
        ('svm', LinearSVC(random_state=42, max_iter=10000)),
        ('lr_l1', LogisticRegression(random_state=42, penalty="l1", solver="liblinear"),
        ('rf'), RandomForestClassifier(random_state=42))]
stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(random_state=42, penalty="l2", solver="liblinear"))

clfs = [MultinomialNB(),
        LogisticRegression(random_state=42, penalty="l1", solver="liblinear"),
        LogisticRegression(random_state=42, penalty="l2", solver="liblinear"),
        LinearSVC(random_state=42, max_iter=10000),
        SVC(random_state=42),
        DecisionTreeClassifier(random_state=42),
        RandomForestClassifier(random_state=42),
        AdaBoostClassifier(random_state=42),
        GradientBoostingClassifier(random_state=42),
        stacking
        ]

In [4]:
from joblib import Parallel, delayed
def process(clf):
    clf_str = clf.__str__()
    print(f"Running ===> {clf_str}")
    evaluation = list()
    clf_list = list()
    usernames = list(np.unique(data["username"]))

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(   
                data, author1, author2)

            author_clf = AuthorClassifier(clf=clf, scaler=MinMaxScaler(), embeddings=True)
            author_clf.fit(X_train, y_train)
            y_pred = author_clf.predict(X_test)
            evaluation.append(author_clf.evaluate(y_test, y_pred))
            clf_list.append(clf_str)
            metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()

    print(f"Finish <=== {clf_str}")
    return (clf_str, metrics, evaluation, clf_list)
        
results = Parallel(n_jobs=6)(delayed(process)(clf) for clf in clfs)

Running ===> MultinomialNB()
Running ===> LogisticRegression(penalty='l1', random_state=42, solver='liblinear')
Running ===> LogisticRegression(random_state=42, solver='liblinear')
Running ===> DecisionTreeClassifier(random_state=42)
Running ===> LinearSVC(max_iter=10000, random_state=42)
Running ===> SVC(random_state=42)
Finish <=== MultinomialNB()
Running ===> RandomForestClassifier(random_state=42)
Finish <=== LogisticRegression(random_state=42, solver='liblinear')
Running ===> AdaBoostClassifier(random_state=42)
Finish <=== DecisionTreeClassifier(random_state=42)
Running ===> GradientBoostingClassifier(random_state=42)


/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-Kyc7Ejcc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Finish <=== SVC(random_state=42)
Running ===> StackingClassifier(estimators=[('svm',
                                LinearSVC(max_iter=10000, random_state=42)),
                               ('lr_l1',
                                LogisticRegression(penalty='l1',
                                                   random_state=42,
                                                   solver='liblinear'),
                                'rf',
                                RandomForestClassifier(random_state=42))],
                   final_estimator=LogisticRegression(random_state=42,
                                                      solver='liblinear'))


/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-Kyc7Ejcc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Finish <=== LogisticRegression(penalty='l1', random_state=42, solver='liblinear')
Finish <=== LinearSVC(max_iter=10000, random_state=42)
Finish <=== RandomForestClassifier(random_state=42)
Finish <=== AdaBoostClassifier(random_state=42)


/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-Kyc7Ejcc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-Kyc7Ejcc/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Finish <=== GradientBoostingClassifier(random_state=42)
Finish <=== StackingClassifier(estimators=[('svm',
                                LinearSVC(max_iter=10000, random_state=42)),
                               ('lr_l1',
                                LogisticRegression(penalty='l1',
                                                   random_state=42,
                                                   solver='liblinear'),
                                'rf',
                                RandomForestClassifier(random_state=42))],
                   final_estimator=LogisticRegression(random_state=42,
                                                      solver='liblinear'))


In [5]:
import itertools
scores = [results[i][2] for i in range(10)]
clfs_list = [results[i][3] for i in range(10)]

clfs_list = list(itertools.chain(*clfs_list))
scores = list(itertools.chain(*scores))

metrics_full = pd.concat([pd.DataFrame(scores), pd.DataFrame(clfs_list, columns=["clf"])], axis=1)
metrics_full.to_csv("../../results/word2vec_results_full.csv")
metrics_full

,author1,author2,precision_author1,recall_author1,f1_score_author1,precision_author2,recall_author2,f1_score_author2,precision_weighted,precision_micro,precision_macro,recall_weighted,recall_micro,recall_macro,f1_weighted,f1_micro,f1_macro,auc_score,accuracy,clf
0,9-Cortes,xanax101010,0.6036,0.9478,0.7375,0.8713,0.3621,0.5116,0.7358,0.6585,0.7374,0.6585,0.6585,0.6550,0.6259,0.6585,0.6246,0.7651,0.6585,MultinomialNB()
1,AgnaldoTeExplode,xanax101010,0.5799,0.9145,0.7098,0.8148,0.3621,0.5014,0.6996,0.6331,0.6974,0.6331,0.6331,0.6383,0.6036,0.6331,0.6056,0.6978,0.6331,MultinomialNB()
2,BeatoSalut,xanax101010,0.4888,0.6245,0.5484,0.4743,0.3416,0.3971,0.4816,0.4836,0.4816,0.4836,0.4836,0.4830,0.4731,0.4836,0.4728,0.5374,0.4836,MultinomialNB()
3,BluePirate89,xanax101010,0.6221,0.4332,0.5107,0.5597,0.7325,0.6346,0.5912,0.5816,0.5909,0.5816,0.5816,0.5829,0.5722,0.5816,0.5727,0.6053,0.5816,MultinomialNB()
4,CariocaSatanico,xanax101010,0.6071,0.9558,0.7426,0.8900,0.3663,0.5190,0.7468,0.6646,0.7486,0.6646,0.6646,0.6610,0.6321,0.6646,0.6308,0.7458,0.6646,MultinomialNB()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,AgnaldoTeExplode,BluePirate89,0.7992,0.8675,0.8320,0.8634,0.7935,0.8270,0.8322,0.8295,0.8313,0.8295,0.8295,0.8305,0.8294,0.8295,0.8295,0.9012,0.8295,"StackingClassifier(estimators=[('svm',\n ..."
1046,BeatoSalut,BluePirate89,0.6892,0.7061,0.6976,0.7012,0.6842,0.6926,0.6953,0.6951,0.6952,0.6951,0.6951,0.6952,0.6951,0.6951,0.6951,0.7573,0.6951,"StackingClassifier(estimators=[('svm',\n ..."
1047,9-Cortes,BeatoSalut,0.7614,0.8715,0.8127,0.8469,0.7224,0.7797,0.8038,0.7976,0.8041,0.7976,0.7976,0.7970,0.7964,0.7976,0.7962,0.8492,0.7976,"StackingClassifier(estimators=[('svm',\n ..."
1048,AgnaldoTeExplode,BeatoSalut,0.7623,0.8632,0.8096,0.8505,0.7429,0.7930,0.8074,0.8017,0.8064,0.8017,0.8017,0.8031,0.8011,0.8017,0.8013,0.8636,0.8017,"StackingClassifier(estimators=[('svm',\n ..."


In [8]:
metrics_df = pd.DataFrame([results[i][1] for i in range(len(results))])
metrics_df["classifier"] = [results[i][0] for i in range(len(results))]
metrics_df

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,classifier
0,0.630535,0.650665,0.690520,0.650862,0.710926,MultinomialNB()
1,0.750673,0.752473,0.759603,0.752688,0.825631,"LogisticRegression(penalty='l1', random_state=..."
2,0.745508,0.747753,0.756851,0.748021,0.822596,"LogisticRegression(random_state=42, solver='li..."
3,0.768187,0.769520,0.775856,0.769770,0.843301,"LinearSVC(max_iter=10000, random_state=42)"
4,0.741613,0.745708,0.759746,0.745841,0.836346,SVC(random_state=42)
5,0.696137,0.696350,0.696900,0.696498,0.696340,DecisionTreeClassifier(random_state=42)
6,0.773769,0.774320,0.775982,0.774233,0.848810,RandomForestClassifier(random_state=42)
7,0.755216,0.755476,0.756490,0.755632,0.826072,AdaBoostClassifier(random_state=42)
8,0.772878,0.773221,0.774362,0.773246,0.848967,GradientBoostingClassifier(random_state=42)
9,0.770671,0.771605,0.775839,0.771796,0.842546,"StackingClassifier(estimators=[('svm',\n ..."


In [9]:
metrics_df.to_csv("../../results/word2vec_results.csv")